Классификация - распространенная задача в сфере обработки текстов. В данном соревновании мы будем классифицировать настоящие данные с сайта Lenta.ru по пяти тематикам:

Экономика

Спорт

Культура

Наука и техника

Бизнес

В данных присутствуют два поля: заголовок (title) и сам текст статьи (text). Вы можете использовать одно из полей или оба для определения класса. Метка класса лежит в поле topic_label.

In [1]:
import pandas as pd
import numpy as np

import urllib.request # библиотека для скачивания данных
import gensim # библиотека для загрузки и использвоания моделй w2v
from gensim.models import word2vec # непосредственно методы w2v

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

import warnings
warnings.filterwarnings("ignore")

import nltk.data # библиотека Natural Language Toolkit
import re   # библиотека для регулярных выражений
from nltk.corpus import stopwords # стоп-слова из NLTK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer  # токенизаторы из NLTK
nltk.download('punkt') # для правильной работы токенизатора

from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nataliakokareva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('lenta-ru-train.csv')
data.shape

(206637, 4)

In [3]:
data_small = data[:100000]
texts = data_small['text'].values
titles = data_small['title'].values
labels = data_small['topic_label'].values

In [4]:
len(texts)

100000

### Токенизация и лемматизация
Каждый раз можно это не запускать. Леммы уже сохранены в файле 'lenta-ru-train-small-lem-tag-text.csv'
См. пункт "загрузка предварительно сохраненных данных

In [16]:
from pymystem3 import Mystem


In [10]:

#text_tokens = []
tmp_lemmas = []
m = Mystem()
for text in texts:
    lem = []
    clean_text = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', text)
    tkn = clean_text.split()
    for t in tkn:
        lem.append(m.lemmatize(t))
    tmp_lemmas.append(lem)
    #text_tokens.append(tkn)



In [11]:
text_lemmas = []
for sent in tmp_lemmas:
    tmp = []
    for word in sent:
        tmp.append(word[0].rstrip('\n'))
    text_lemmas.append(' '.join(tmp))
        

In [14]:
m = Mystem()
lem_tag = []
for text in text_lemmas:
    p = m.analyze(text)
    tmp = []
    for word in p:
        if 'analysis' in word:
            if word['analysis'] != []:
                tup = (word['analysis'][0]['lex'], word['analysis'][0]['gr'].split('=')[0].split(',')[0])
                tmp.append('_'.join(tup))
    lem_tag.append(' '.join(tmp))
lem_tag

['глава_S росстат_S владимир_S соколин_S заявлять_V что_CONJ в_PR январь_S год_S темп_S рост_S цена_S в_PR россия_S составлять_V процент_S то_CONJ быть_V столько_NUM же_PART сколько_CONJ они_SPRO составлять_V в_PR январь_S прошлое_S год_S писать_V ведомость_S тогда_ADVPRO как_ADVPRO и_CONJ в_PR год_S правительство_S планировать_V удерживать_V инфляция_S в_PR предел_S процент_S однако_CONJ в_PR год_S прогноз_S рост_S цена_S приходиться_V дважды_ADV пересматривать_V сначала_ADV до_PR а_CONJ затем_ADV до_PR процент_S кроме_PR то_SPRO чиновник_S отчитываться_V за_PR год_S в_PR качество_S основной_A причина_S высокий_A темп_S рост_S цена_S в_PR год_S указывать_V именно_PART на_PR январский_A инфляция_S в_PR частность_S на_PR повышение_S в_PR январь_S тариф_S на_PR услуга_S жкх_S увеличение_S бюджетный_A расход_S в_PR связь_S с_PR монетизация_S льгота_S и_CONJ рост_S цена_S на_PR бензин_S причина_S нынешний_A инфляция_S выясняться_V в_PR февраль_S когда_CONJ появляться_V официальный_A статис

In [15]:
data_small['lem_tag'] = lem_tag
data_small.head()

,title,text,topic,topic_label,lem_tag
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0,глава_S росстат_S владимир_S соколин_S заявлят...
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3,у_PR российский_A кинематографист_S появляться...
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1,медицинский_A штаб_S миланский_A интер_S обнар...
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3,премия_S книга_S год_S в_PR номинация_S вручат...
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1,сборная_S португалия_S сыграть_V вничью_ADV с_...


In [16]:
data_small.to_csv('lenta-ru-train-small-lem-tag-text.csv')

#### Загрузка предварительно сохраненных данных

In [5]:
data_new = pd.read_csv('lenta-ru-train-small-lem-tag-text.csv')
data_new.head()

,Unnamed: 0,title,text,topic,topic_label,lem_tag
0,0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0,глава_S росстат_S владимир_S соколин_S заявлят...
1,1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3,у_PR российский_A кинематографист_S появляться...
2,2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1,медицинский_A штаб_S миланский_A интер_S обнар...
3,3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3,премия_S книга_S год_S в_PR номинация_S вручат...
4,4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1,сборная_S португалия_S сыграть_V вничью_ADV с_...


In [6]:
data_new = data_new.replace(np.nan, '', regex=True)

In [7]:
lst = data_new['lem_tag'].values
lem_tag_read = []
for i in range(len(lst)):
    lem_tag_read.append(lst[i])


In [8]:
model_path = "ruscorpora_mean_hs.model.bin"
model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [9]:
len(model_ru['глава_S'])

300

#### Средние эмбеддинги по всем словам отзыва

In [10]:
lem_tag_lst = []
for i in range(len(lem_tag_read)):
    lem_tag_lst.append(lem_tag_read[i].split())


In [10]:
mean_vec = []
for i in range(len(lem_tag_lst)):
    vec = np.zeros(300)
    for word in lem_tag_lst[i]:
            try:
                vec += model_ru[word]
            except:
                vec = vec
    mean_vec.append(vec / len(lem_tag_lst[i]))

In [11]:
len(mean_vec)

100000

In [12]:
X = []
y = []
for i in range(len(mean_vec)):
    if np.isnan(mean_vec[i]).any() == True:
        continue
    X.append(mean_vec[i])
    y.append(labels[i])
        

In [13]:
X = np.array(X)
y = np.array(y)

In [46]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [16]:
for clf in [LogisticRegression(), LinearSVC(), SGDClassifier()]:
    print(clf)
    print(cross_val_score(clf, X, y).mean())
    print("\n")

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.9254007391670612


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)
0.9386294880717415


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=

#### Подбор параметров:

In [18]:
#LogisticRegression

grid_lr = {
    'C': np.logspace(-3,3,7),
    'penalty': ['l1', 'l2']
}

сlf_lr = LogisticRegression()
clf_lr_cv=GridSearchCV(сlf_lr,grid_lr,cv=10)
clf_lr_cv.fit(X,y)

print("LogisticRegression:")
print(f"Лучшее качество - {clf_lr_cv.best_score_}")
print(f"Параметры - {clf_lr_cv.best_params_}")

LogisticRegression:
Лучшее качество - 0.9431524806372206
Параметры - {'C': 1000.0, 'penalty': 'l1'}


In [19]:
#LinearSVC

grid_lsvc = {
    'loss': ['hinge', 'squared_hinge'], 
    'C': np.logspace(-3,3,7)
}

clf_lsvc = LinearSVC()
clf_lsvc_cv=GridSearchCV(clf_lsvc,grid_lsvc,cv=10)
clf_lsvc_cv.fit(X,y)

print("LinearSVC:")
print(f"Лучшее качество - {clf_lsvc_cv.best_score_}")
print(f"Параметры - {clf_lsvc_cv.best_params_}")

LinearSVC:
Лучшее качество - 0.9428022494846598
Параметры - {'C': 100.0, 'loss': 'squared_hinge'}


In [20]:
#SGDClassifier

grid_sgd = {
    'loss': ['log', 'hinge', 'modified_huber'], 
    'penalty':  ['l1', 'l2', 'elasticnet']
}

clf_sgd = SGDClassifier()
clf_sgd_cv=GridSearchCV(clf_sgd,grid_sgd,cv=10)
clf_sgd_cv.fit(X,y)

print("SGDClassifier:")
print(f"Лучшее качество - {clf_sgd_cv.best_score_}")
print(f"Параметры - {clf_sgd_cv.best_params_}")

SGDClassifier:
Лучшее качество - 0.9335361338483399
Параметры - {'loss': 'modified_huber', 'penalty': 'l1'}


#### Предобработка тестовых данных

In [26]:
data_test = pd.read_csv('lenta-ru-test.csv')

Index(['title', 'text', 'topic', 'topic_label'], dtype='object')

In [39]:
texts_test = data_test['text'].values
titles_test = data_test['title'].values

In [40]:
tmp_lemmas_test = []
m = Mystem()
for text in texts_test:
    lem = []
    clean_text = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', text)
    tkn = clean_text.split()
    for t in tkn:
        lem.append(m.lemmatize(t))
    tmp_lemmas_test.append(lem)

In [41]:
text_lemmas_test = []
for sent in tmp_lemmas_test:
    tmp = []
    for word in sent:
        tmp.append(word[0].rstrip('\n'))
    text_lemmas_test.append(' '.join(tmp))
        

In [42]:
m = Mystem()
lem_tag_test = []
for text in text_lemmas_test:
    p = m.analyze(text)
    tmp = []
    for word in p:
        if 'analysis' in word:
            if word['analysis'] != []:
                tup = (word['analysis'][0]['lex'], word['analysis'][0]['gr'].split('=')[0].split(',')[0])
                tmp.append('_'.join(tup))
    lem_tag_test.append(' '.join(tmp))
lem_tag_test[:3]

['в_PR южноафриканский_A кейптаун_S победа_S сборная_S россия_S завершаться_V чемпионат_S мир_S среди_PR бездомный_S в_PR финальный_A матч_S российский_A футболист_S впервые_ADV в_PR свой_APRO история_S становиться_V чемпион_S мир_S обыгрывать_V команда_S казахстан_S со_PR счет_S передавать_V в_PR первенство_S принимать_V участие_S почти_ADV человек_S который_APRO представлять_V страна_S мир_S весь_APRO матч_S каждый_APRO из_PR который_APRO продолжаться_V минута_S проходить_V на_PR асфальтовый_A поле_S причем_CONJ в_PR один_APRO команда_S мочь_V играть_V как_CONJ мужчина_S так_ADVPRO и_CONJ женщина_S сборная_S россия_S проводить_V на_PR турнир_S матч_S во_PR весь_APRO из_PR который_APRO добиваться_V победа_S на_PR предыдущий_A чемпионат_S мир_S достижение_S российский_A команда_S быть_V скромный_A в_PR м_S год_S е_S место_S в_PR м_S год_S е_S место_S в_PR м_S год_S е_S место_S',
 'анна_S курникова_S проигрывать_V испанка_S кончать_V мартинес_S уже_ADV в_PR первый_ANUM круг_S выбывать_V

In [43]:
lem_tag_lst_test = []
for i in range(len(lem_tag_test)):
    lem_tag_lst_test.append(lem_tag_test[i].split())


In [44]:
mean_vec_test = []
for i in range(len(lem_tag_lst_test)):
    vec = np.zeros(300)
    for word in lem_tag_lst_test[i]:
            try:
                vec += model_ru[word]
            except:
                vec = vec
    mean_vec_test.append(vec / len(lem_tag_lst_test[i]))

In [57]:
X_test = []
for i in range(len(mean_vec_test)):
    if np.isnan(mean_vec_test[i]).any() == True:
        continue
    X_test.append(mean_vec_test[i])

#### Предсказания

In [58]:
# LogisticRegression. Параметры - {'C': 1000.0, 'penalty': 'l1'}

clf_lr = LogisticRegression(C = 1000, penalty = 'l1')
clf_lr.fit(X, y)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [59]:
y_pred_lr = clf.predict(X_test)
y_pred_lr

array([1, 1, 1, ..., 2, 1, 1])

In [62]:
# LinearSVC Параметры - {'C': 100.0, 'loss': 'squared_hinge'}
clf_lsvc = LinearSVC(C = 100, loss = 'squared_hinge')
clf_lsvc.fit(X, y)
y_pred_lsvc = clf_lsvc.predict(X_test)

In [63]:
y_pred_lsvc

array([1, 1, 1, ..., 2, 1, 1])

In [71]:
# SGDClassifier Параметры - {'loss': 'modified_huber', 'penalty': 'l1'}
clf_sgd = SGDClassifier(loss = 'modified_huber', penalty = 'l1')
clf_sgd.fit(X, y)
y_pred_sgd = clf_sgd.predict(X_test)
y_pred_sgd

array([1, 1, 1, ..., 2, 1, 1])